# TVR Layout Editing Playground

Этот блокнот показывает полный цикл:

1. загрузка исходного TVR и парсинг выбранных стратегий;
2. получение структуры в относительных `begin+N` индексах;
3. ручное изменение структуры;
4. генерация нового фрагмента TVR с проверками.

## 0. Настройки
Заполните параметры перед запуском остальных ячеек.

In [3]:
from pathlib import Path

from src.tvr_service.generator import (
    build_layout_from_source,
    parse_layout_text,
    compile_layout,
    generate_robot_segment,
)

# Путь к исходному TVR-файлу
SOURCE_TVR = Path("Default.tvr2")

# Список базовых строк (long/short), которые хотим спарсить
BASE_STROKAS = [5562, 5566]

# Базовое назначение begin-позиций (где должна оказаться каждая база)
BASE_ASSIGNMENT = {5562: 1, 5566: 5}

# Основные параметры генерации
SEC0_VALUE = "ММЕ55"  # Пример: заменить инструмент
OUTPUT_TVR_PATH = Path("generated_from_layout.tvr2")
OUTPUT_EXCEL_PATH = Path("generated_from_layout.xlsx")  # Поставьте None, если Excel не нужен

# Если True, то при парсинге дублируем общие фильтры (обычно оставляем False)
DUPLICATE_SHARED_FILTERS = False

## 1. Получаем структуру шаблона
Запускаем парсер и смотрим структуру относительно базы (0 = первая база).

In [4]:
layout = build_layout_from_source(
    str(SOURCE_TVR),
    BASE_STROKAS,
    duplicate_shared_filters=DUPLICATE_SHARED_FILTERS,
)

layout_lines = layout.to_lines()
for line in layout_lines:
    print(line)
print()
print(f"Всего строк: {len(layout_lines)}")


#base_long: 0
##filter_1_long_&_short: 8
###filter_2_long_&_short: 12
####filter_3_long_&_short: 13
#####filter_4_long_&_short: 14
######filter_5_long_&_short: 16
#######filter_6_long_&_short: 17
########filter_7_long_&_short: 18
####filter_3_long_&_short: 15
#base_short: 4

Всего строк: 10


## 2. Готовим текст для редактирования
Ячейка ниже создаёт строку `edited_text`. Скопируйте её в буфер, отредактируйте прямо в ячейке и запустите заново.

* Формат строки: `#label: текущий_offset[: новый_offset]`.
* Если второй offset не задан, позиция не меняется.

In [ ]:


edited_text = """#base_long: 0
##filter_1_long_&_short: 8 : 3
###filter_2_long_&_short: 12 : 4
####filter_3_long_&_short: 13 : 5
#####filter_4_long_&_short: 14 : 6
######filter_5_long_&_short: 16 : 8
#######filter_6_long_&_short: 17 : 9
########filter_7_long_&_short: 18 : 10
####filter_3_long_&_short: 15 : 7
#base_short: 4 : 1"""





После того как внесли изменения и перезапустили ячейку с `edited_text`, выполните код ниже для проверки синтаксиса.

In [13]:
edits = parse_layout_text(layout, edited_text)
print("Обнаружены изменения (stroka -> новый относительный offset):")
print(edits.relative_overrides if edits.relative_overrides else "нет изменений")

Обнаружены изменения (stroka -> новый относительный offset):
{5570: 3, 5574: 4, 5575: 5, 5576: 6, 5578: 8, 5579: 9, 5580: 10, 5577: 7, 5566: 1}


## 3. Компилируем новую структуру
Получаем `base_assignment` и `stroka_overrides`, которые потом передадим генератору.

In [14]:
compiled_layout = compile_layout(layout, edits, BASE_ASSIGNMENT)
print("base_assignment ->", compiled_layout.base_assignment)
print("stroka_overrides ->", compiled_layout.stroka_overrides if compiled_layout.stroka_overrides else "{}")

base_assignment -> {5562: 1, 5566: 2}
stroka_overrides -> {5570: 4, 5574: 5, 5575: 6, 5576: 7, 5578: 9, 5579: 10, 5580: 11, 5577: 8}


## 4. Генерируем новый фрагмент TVR
Файлы сохраняются по путям, указанным в настройках. DataFrame выводится для быстрого контроля.

In [16]:
result = generate_robot_segment(
    source_tvr_path=str(SOURCE_TVR),
    base_strokas=BASE_STROKAS,
    base_assignment=compiled_layout.base_assignment,
    sec0_value=SEC0_VALUE,
    output_tvr_path=str(OUTPUT_TVR_PATH),
    output_excel_path=str(OUTPUT_EXCEL_PATH) if OUTPUT_EXCEL_PATH is not None else None,
    duplicate_shared_filters=DUPLICATE_SHARED_FILTERS,
    stroka_overrides=compiled_layout.stroka_overrides,
)

print("Готово. Файлы сохранены:")
print("  TVR:", OUTPUT_TVR_PATH.resolve())
if OUTPUT_EXCEL_PATH is not None:
    print("  Excel:", OUTPUT_EXCEL_PATH.resolve())

print("Первые строки результата:")
display(result.dataframe.head())
print("Финальное сопоставление строк (original -> new):")
print(result.stroka_mapping)

c:\Users\user\Documents\piranha\constructor_TVR\converters.py:166: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  wide[c] = pd.to_numeric(wide[c], errors='ignore', downcast='integer')


Готово. Файлы сохранены:
  TVR: C:\Users\user\Documents\piranha\constructor_TVR\generated_from_layout.tvr2
  Excel: C:\Users\user\Documents\piranha\constructor_TVR\generated_from_layout.xlsx
Первые строки результата:


,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,Limit,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
0,1,True,None,None,4,None,8,5,None,None,...,09:00:20-23:50:00,09:00:20-23:50:00,0.0,None,3/0,None,None,0.005,None,None
1,2,True,None,None,4,None,8,5,None,None,...,09:00:20-23:50:00,09:00:20-23:50:00,0.0,None,3/0,None,None,0.005,None,None
2,4,None,None,None,5,None,None,None,None,None,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,None,None,None,None,NaN,None,None
3,5,None,None,None,6,None,8,None,None,None,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,3/1,None,None,None,NaN,None,None
4,6,None,None,None,7,None,None,None,None,None,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,5,None,None,None,NaN,None,None


Финальное сопоставление строк (original -> new):
{5562: 1, 5570: 4, 5574: 5, 5575: 6, 5576: 7, 5578: 9, 5579: 10, 5580: 11, 5577: 8, 5566: 2}
